In [1]:
import pandas as pd # loadingData
from nltk.stem import PorterStemmer # func1,cleanData
from nltk.corpus import stopwords # func1
import regex as re # func1
import nlpaug.augmenter.word as naw #func5 CreateOversamplingWithDataAugmentation
import gc # func5
import numpy as np # (*func7*) one_hot_encode
from nltk import PorterStemmer, word_tokenize
from nltk.corpus import stopwords
import nlpaug.augmenter.word as naw
import tqdm

In [2]:
DataAugmentation, DataAugThreshold = True, 30000
DataFilePath, DataFileName, FileType = "Data/powershell/", "Issueazure-powershellWebScrap", ".csv"
train_test_split = .8

In [3]:
totalData = pd.read_csv(DataFilePath + DataFileName + FileType, error_bad_lines=False, index_col=False, dtype='unicode',
                        encoding='latin-1', low_memory=False).sample(frac=1)
# 替换RepoID列，不在源文件上改动(IssueaspnetcoreWebScrap的列RepoID本来就是RepoID了)
totalData = totalData.rename(columns={'ï»¿RepoID': 'RepoID'}, inplace=False)
# 04/09/2017  -> 2014-01-07  
totalData['CreatedDate'] = pd.to_datetime(totalData['CreatedDate'])
totalData = totalData.sort_values(by=['CreatedDate'])
totalData

,RepoID,IssueID,Title_Description,AST,FixedByID,Name,CreatedDate
1094,azure-powershellWebScrap,558,new-azurereservedip no longer working with pow...,NaN,unknown,network| service attention,2015-01-07
2153,azure-powershellWebScrap,832,support medialink in set-azureplatformextensio...,NaN,singhkays,compute| service attention,2015-01-09
1271,azure-powershellWebScrap,1386,get-azurermnetworksecuritygroup erroraction is...,NaN,unknown,network| service attention,2015-01-12
1269,azure-powershellWebScrap,1384,azurerm cmdlets return object with a json-like...,NaN,singhkays,compute| network| service attention,2015-01-12
1268,azure-powershellWebScrap,1380,setting reversefqdn in new-azurermpublicipaddr...,NaN,unknown,network| service attention,2015-01-12
...,...,...,...,...,...,...,...
1249,azure-powershellWebScrap,11310,azure powershell set-azkeyvaultaccesspolicy co...,NaN,unknown,unknown,2020-12-03
2229,azure-powershellWebScrap,11882,update-azwebappaccessrestrictionconfig -scmsit...,NaN,unknown,app services| service attention| question,2020-12-05
2230,azure-powershellWebScrap,11885,get-azaks is missing (many) details provided b...,NaN,erich-wang,aks| azure ps team| feature-request,2020-12-05
14,azure-powershellWebScrap,12136,hi hi i am having the same issue when i disabl...,NaN,unknown,needs-triage| question,2020-12-06


In [4]:
# Cross Validation in TimeSeries
# PowerShell 2540 508 2032
totalNoOfRecords = len(totalData)
split = int(totalNoOfRecords / 5)  
train_test_split = int(train_test_split * totalNoOfRecords)
print(totalNoOfRecords, split, train_test_split)

2540 508 2032


In [5]:
# cleanData
# 将title_description更新为只保留大小写字母，分词， 去停用词， 去掉单个字母，提取词干后
data = totalData
porter = PorterStemmer() # 词干提取器 eg. coding -> code
for ind in data.index:   # index是默认的4466，4579，...
    # 找到ind的题目描述，只保留字母，word_tokenize分词，去掉stopwords，去掉单字母，提取词干，替换ind的题目描述
    bug_description = data['Title_Description'][ind]
    bug_description = re.sub("[^a-zA-Z]", " ", str(bug_description)) #仅保留字母 /re.sub()
    if bug_description and bug_description.strip():
        tk_bug_description = word_tokenize(bug_description) # 分词 /word_tokenize 
        tokens_without_sw = str(
            [word for word in tk_bug_description if not word in stopwords.words('english')]).strip('[]').replace(
            "'", "").replace(",", "").replace("\\", "") # 去掉 /stopwords 
        tokens_without_sl = ' '.join(
            [w for w in tokens_without_sw.split() if len(w) > 1])  # token without single letter
        tokens_with_stemm = ' '.join([porter.stem(w) for w in tokens_without_sl.split()])  # token with stemm
        data['Title_Description'][ind] = tokens_with_stemm
data[:train_test_split].to_csv(DataFilePath + "C_uA_Train.csv")
data[train_test_split:].to_csv(DataFilePath + "C_uA_Test.csv")
data

/tmp/ipykernel_4984/2793849500.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Title_Description'][ind] = tokens_with_stemm


,RepoID,IssueID,Title_Description,AST,FixedByID,Name,CreatedDate
1094,azure-powershellWebScrap,558,new azurereservedip longer work powershel erro...,NaN,unknown,network| service attention,2015-01-07
2153,azure-powershellWebScrap,832,support medialink set azureplatformextens curr...,NaN,singhkays,compute| service attention,2015-01-09
1271,azure-powershellWebScrap,1386,get azurermnetworksecuritygroup erroract ignor...,NaN,unknown,network| service attention,2015-01-12
1269,azure-powershellWebScrap,1384,azurerm cmdlet return object json like format ...,NaN,singhkays,compute| network| service attention,2015-01-12
1268,azure-powershellWebScrap,1380,set reversefqdn new azurermpublicipaddress ord...,NaN,unknown,network| service attention,2015-01-12
...,...,...,...,...,...,...,...
1249,azure-powershellWebScrap,11310,azur powershel set azkeyvaultaccesspolici comm...,NaN,unknown,unknown,2020-12-03
2229,azure-powershellWebScrap,11882,updat azwebappaccessrestrictionconfig scmsiteu...,NaN,unknown,app services| service attention| question,2020-12-05
2230,azure-powershellWebScrap,11885,get azak miss mani detail provid az ak list ma...,NaN,erich-wang,aks| azure ps team| feature-request,2020-12-05
14,azure-powershellWebScrap,12136,hi hi issu disabl daili backup use weekli mont...,NaN,unknown,needs-triage| question,2020-12-06


In [8]:
# CreateOversamplingWithDataAugmentation
# 将少数列的行数通过bert上下文替换增加到和多数列一样多

# data = totalData # 不cleandata时
traindata = data[:train_test_split]
data = traindata

# /*Get Contextual Word Embedding Model*/
TRANSFORMERS_OFFLINE=1
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute") # aug是上下文替换器
# /*Get Developer Bug Count/
dfcountbybug = data.groupby(["Name", "FixedByID"], as_index=True)["FixedByID"].size().reset_index(name="count")
dfcountbybug.to_csv(DataFilePath + 'list.csv')
# /*Get Majority Class List*/
majoritycount = dfcountbybug[(dfcountbybug['FixedByID'] != 'unknown') & (dfcountbybug['Name'] != 'unknown')][
    'count'].max()
# /*Get Minority Class Count*/
minoritylist = dfcountbybug[(dfcountbybug['count'] != dfcountbybug['count'].max())]
# 42 643 27006
print(majoritycount, len(minoritylist), majoritycount * len(minoritylist))
estimatetotalnoofdataaugrecord = majoritycount * len(minoritylist)
maxnoofaug = majoritycount
## Data Aug Record Count Validation, If over threshold reduce the majaritycount
if estimatetotalnoofdataaugrecord > DataAugThreshold: # 30000
    print('Overtheshold--')
    maxnoofaug = int((DataAugThreshold / estimatetotalnoofdataaugrecord) * majoritycount)

# /*Loop through Minor Class Group*/
for ind in tqdm.tqdm(minoritylist.index):
    # print(minoritylist['FixedByID'][ind], minoritylist['count'][ind])
    developer = minoritylist['FixedByID'][ind]
    bugtype = minoritylist['Name'][ind]
    minoritycount = minoritylist['count'][ind]
    data1 = data[(data['FixedByID'] == developer) & (data['Name'] == bugtype)]
    # print(len(data1), developer,bugtype)
    # print('minoritycount  --->',minoritycount, 'majoritycount--->',majoritycount, 'index --->', ind , 'out of ', len(minoritylist.index))
    # Create Sample Data until minority class count Match up with majority class count
    while minoritycount < maxnoofaug:
        # majoritycount:
        samplerow = data1.sample()
        oldbugdescription = str(samplerow['Title_Description'].values).strip('[]').replace("'", "")
        if oldbugdescription:
            first100words_aug = str(aug.augment(' '.join(oldbugdescription.split()[:100])))
            remainingwords = ' '.join(oldbugdescription.split()[100:])
            newbugdescription = first100words_aug + remainingwords
            new_row = {'RepoID': str(samplerow['RepoID'].values).strip('[]').replace("'", ""),
                        # 'PullRequestID' : str(samplerow['PullRequestID'].values).strip('[]').replace("'",""),
                       'IssueID': str(samplerow['IssueID'].values).strip('[]').replace("'", ""),
                       'Title_Description': newbugdescription,  # /*Data Augmentation : Title_Desciption*/
                       'AST': str(samplerow['AST'].values).strip('[]').replace("'", ""),
                       'FixedByID': str(samplerow['FixedByID'].values).strip('[]').replace("'", ""),
                       'Name': str(samplerow['Name'].values).strip('[]').replace("'", ""),
                       'CreatedDate': str(samplerow['CreatedDate'].values).strip('[]').replace("'", "")}
            data = data.append(new_row, ignore_index=True)
            minoritycount = minoritycount + 1
        gc.collect()

data.to_csv(DataFilePath + 'C_A_Train.csv')
data

42 643 27006


100%|██████████| 643/643 [2:53:59<00:00, 16.24s/it]  


,RepoID,IssueID,Title_Description,AST,FixedByID,Name,CreatedDate
0,azure-powershellWebScrap,558,new azurereservedip longer work powershel erro...,NaN,unknown,network| service attention,2015-01-07 00:00:00
1,azure-powershellWebScrap,832,support medialink set azureplatformextens curr...,NaN,singhkays,compute| service attention,2015-01-09 00:00:00
2,azure-powershellWebScrap,1386,get azurermnetworksecuritygroup erroract ignor...,NaN,unknown,network| service attention,2015-01-12 00:00:00
3,azure-powershellWebScrap,1384,azurerm cmdlet return object json like format ...,NaN,singhkays,compute| network| service attention,2015-01-12 00:00:00
4,azure-powershellWebScrap,1380,set reversefqdn new azurermpublicipaddress ord...,NaN,unknown,network| service attention,2015-01-12 00:00:00
...,...,...,...,...,...,...,...
27401,azure-powershellWebScrap,4436,['sidebysid creat and side msi build azurestac...,nan,viananth,versioning,Timestamp(2017-09-08 00:00:00)
27402,azure-powershellWebScrap,4436,['c creat side development msi version azurest...,nan,viananth,versioning,Timestamp(2017-09-08 00:00:00)
27403,azure-powershellWebScrap,4436,['sidebysid creat side side x build start clie...,nan,viananth,versioning,Timestamp(2017-09-08 00:00:00)
27404,azure-powershellWebScrap,4436,['sidebysid creat the gui to build azurestack ...,nan,viananth,versioning,Timestamp(2017-09-08 00:00:00)
